In [1]:
!pip install -q openai langchain langchain-community tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [3]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [10]:
!pip install -q langchain-openai langchain-tavily

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 30.7 MB/s eta 0:00:00


In [11]:
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.tools import Tool as CoreTool
from langchain_core.prompts import ChatPromptTemplate

# Initialize GPT model
llm = ChatOpenAI(
    model="gpt-3.5-turbo-1106",  # or "gpt-4" if available
    temperature=0.7,
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

# Initialize Tavily search tool
search_tool = TavilySearch(api_key=os.environ["TAVILY_API_KEY"])

# Define tool
tools = [
    CoreTool.from_function(
        name="search",
        func=search_tool.run,
        description="Useful for answering questions by searching the web."
    )
]

# Prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful research assistant."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Create agent
agent = create_tool_calling_agent(llm, tools, prompt_template)

# Agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [12]:
topic = "Artificial Intelligence in Healthcare"

prompt = f"""
You are a research agent. Your goal is to collect information on the topic: "{topic}".

Step 1: Generate 5-6 research questions covering different aspects of the topic.
Step 2: For each question, search the web using the search tool.
Step 3: Extract relevant points and compile a structured report with title, intro, sections per question, and conclusion.

Start with generating the questions.
"""

response = agent_executor.invoke({"input": prompt})
print(response["output"])




> Entering new AgentExecutor chain...

Invoking: `search` with `How is artificial intelligence used in medical imaging and diagnostics?`
responded: Sure, I can help with that. Let's start by generating 5-6 research questions on the topic "Artificial Intelligence in Healthcare."

1. How is artificial intelligence used in medical imaging and diagnostics?
2. What are the ethical considerations and challenges associated with the use of AI in healthcare?
3. How does AI contribute to personalized medicine and treatment?
4. What are the current and potential future applications of AI in healthcare research and development?
5. How does AI improve patient care and clinical decision-making in healthcare?
6. What are the limitations and risks of implementing artificial intelligence in healthcare settings?

These questions cover various aspects of artificial intelligence in healthcare, including its applications, ethical considerations, impact on patient care, and potential limitations. Now, I w